<a href="https://colab.research.google.com/github/jatchison13/Neuroscience-Data/blob/master/VDR_Decoding_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, style="whitegrid", palette="pastel", color_codes=True)

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_predict, LeaveOneOut
import os
import csv

In [2]:
##Download data file, set name

!wget https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
!mv 136_Data_Final.csv\?raw\=true 136_Data_Final.csv

--2019-04-15 17:50:44--  https://github.com/blelliott23/neuro-data-analysis-course/blob/master/VDR_Decode/Data/136/136_Data_Final.csv?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-15 17:50:49--  https://github.com/blelliott23/neuro-data-analysis-course/raw/master/VDR_Decode/Data/136/136_Data_Final.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv [following]
--2019-04-15 17:50:49--  https://raw.githubusercontent.com/blelliott23/neuro-data-analysis-course/master/VDR_Decode/Data/136/136_Data_Final.csv
Resolving raw.githubus

In [3]:
#Import data to dataframe
##Implement Random forest stuff here...
df = pd.read_csv('136_Data_Final.csv')
df.set_index('Subject')
df.head()

Unnamed: 0  Subject  Block  Trial  Time      O2      O1      OZ      PZ  \
0           0      136      1      1  -200 -2.6587 -5.8283 -3.6200 -8.4912   
1           1      136      1      1  -196 -3.3376 -5.6611 -3.5900 -8.8003   
2           2      136      1      1  -192 -3.4338 -4.6322 -2.9857 -8.3836   
3           3      136      1      1  -188 -3.6411 -3.6174 -2.5697 -7.9997   
4           4      136      1      1  -184 -3.9098 -3.0866 -2.5453 -8.4382   

       P4  ...       FC3      F3     FP2      F7     FP1      VEOU     VEOL  \
0 -6.0398  ...   -4.2465 -6.1961 -0.7449 -4.0477 -6.5453 -252.5419  70.5294   
1 -6.9963  ...   -3.2880 -5.5053 -0.7434 -3.1528 -4.8430 -218.5595  51.3878   
2 -7.6261  ...   -2.0604 -4.9363 -0.8046 -2.6793 -2.9363 -196.0244  68.5147   
3 -8.4763  ...   -1.1900 -5.0391 -0.8101 -2.7765 -1.7893 -191.9580  51.4932   
4 -9.8238  ...   -0.9082 -5.5007 -0.6975 -3.0212 -2.2171 -177.3466  41.4210   

      HEOR     HEOL  Value  
0  13.6076 -66.9374    1.0  
1  25.2795 -68.4394    1.0  
2  18.5178 -64.9081    1.0  
3  22.3986 -66.2655    1.0  
4  21.8185 -66.5214    1.0  

[5 rows x 40 columns]

In [0]:
df_one_hot_encoded = pd.get_dummies(df)
df = df_one_hot_encoded

In [5]:
df_1 = df.groupby(np.arange(len(df))//10).mean()
df_1.head(60)

Unnamed: 0  Subject  Block  Trial    Time        O2        O1        OZ  \
0          4.5    136.0    1.0    1.0  -182.0  -4.30124  -2.44688  -2.91630   
1         14.5    136.0    1.0    1.0  -142.0   3.05864   4.78650   3.70634   
2         24.5    136.0    1.0    1.0  -102.0   3.08058   1.19648   2.24625   
3         34.5    136.0    1.0    1.0   -62.0   5.42963  -0.22368   2.65670   
4         44.5    136.0    1.0    1.0   -22.0  -7.26760  -3.31240  -5.69298   
5         54.5    136.0    1.0    1.0    18.0 -13.54864 -14.35524 -13.06082   
6         64.5    136.0    1.0    1.0    58.0 -11.59703  -3.99237  -7.01326   
7         74.5    136.0    1.0    1.0    98.0  -9.36283  -0.73690  -5.33622   
8         84.5    136.0    1.0    1.0   138.0  -5.90463  -2.25643  -4.24257   
9         94.5    136.0    1.0    1.0   178.0  -9.35645  -0.82725  -4.86715   
10       104.5    136.0    1.0    1.0   218.0  -3.64236  -1.23297  -2.78691   
11       114.5    136.0    1.0    1.0   258.0 -14.28126  -8.53094 -13.25681   
12       124.5    136.0    1.0    1.0   298.0 -12.10998  -3.82831 -10.86193   
13       134.5    136.0    1.0    1.0   338.0  -0.65833  10.03850   2.11435   
14       144.5    136.0    1.0    1.0   378.0  -6.50856   9.57284   0.01985   
15       154.5    136.0    1.0    1.0   418.0 -15.74794   2.52653  -9.71895   
16       164.5    136.0    1.0    1.0   458.0 -15.61725   3.80189  -7.88741   
17       174.5    136.0    1.0    1.0   498.0 -10.89015   7.86756  -4.21207   
18       184.5    136.0    1.0    1.0   538.0 -20.94369   1.53176 -11.92323   
19       194.5    136.0    1.0    1.0   578.0 -29.65594  -5.60801 -20.85366   
20       204.5    136.0    1.0    1.0   618.0 -16.64471   5.32631  -8.98673   
21       214.5    136.0    1.0    1.0   658.0 -18.27105   4.00176  -9.67293   
22       224.5    136.0    1.0    1.0   698.0 -23.32239   0.60909 -14.05149   
23       234.5    136.0    1.0    1.0   738.0 -15.53720   3.47710 -10.37850   
24       244.5    136.0    1.0    1.0   778.0 -28.65981  -2.66798 -19.49635   
25       254.5    136.0    1.0    1.0   818.0 -35.76948  -9.72890 -25.62106   
26       264.5    136.0    1.0    1.0   858.0 -34.70347  -9.62430 -24.94393   
27       274.5    136.0    1.0    1.0   898.0 -38.52843 -11.81415 -27.78656   
28       284.5    136.0    1.0    1.0   938.0 -25.00287  -0.12264 -15.64832   
29       294.5    136.0    1.0    1.0   978.0 -24.57550   2.57855 -13.61342   
30       304.5    136.0    1.0    1.0  1018.0 -28.09275   4.47902 -15.59163   
31       314.5    136.0    1.0    1.0  1058.0 -37.98808 -12.63889 -27.12394   
32       324.5    136.0    1.0    1.0  1098.0 -39.94056  -7.25380 -26.88969   
33       334.5    136.0    1.0    1.0  1138.0 -35.67217  -7.46376 -24.79981   
34       344.5    136.0    1.0    1.0  1178.0 -36.29826  -5.28242 -22.82559   
35       354.5    136.0    1.0    1.0  1218.0 -29.58844  -0.98427 -16.94433   
36       364.5    136.0    1.0    1.0  1258.0 -34.54299  -5.60720 -22.59691   
37       374.5    136.0    1.0    1.0  1298.0 -38.04350  -6.44108 -26.58723   
38       384.5    136.0    1.0    1.0  1338.0 -28.51111   2.78804 -15.91645   
39       394.5    136.0    1.0    1.0  1378.0 -29.22071  -0.23458 -17.16830   
40       404.5    136.0    1.0    1.0  1418.0 -38.31994  -5.68239 -24.14543   
41       414.5    136.0    1.0    1.0  1458.0 -37.40545  -6.47622 -23.70817   
42       424.5    136.0    1.0    1.0  1498.0 -37.32228  -1.97149 -19.97960   
43       434.5    136.0    1.0    1.0  1538.0 -38.87300  -8.16060 -23.53697   
44       444.5    136.0    1.0    1.0  1578.0 -31.02670   2.24849 -15.53531   
45       454.5    136.0    1.0    1.0  1618.0 -41.54016  -4.40801 -25.40363   
46       464.5    136.0    1.0    1.0  1658.0 -42.63088  -7.88108 -26.98038   
47       474.5    136.0    1.0    1.0  1698.0 -39.80199  -1.00275 -21.65905   
48       484.5    136.0    1.0    1.0  1738.0 -38.04266  -3.21919 -21.42116   
49       494.5    136.0    1.0    1.0  1778.0 -42.9013

In [0]:
rfc = RandomForestClassifier(n_estimators=50, max_features=34)
#rfc.fit(x_pd, y_pd);

In [0]:
from sklearn.model_selection import GroupShuffleSplit
cv = GroupShuffleSplit(n_splits=5, test_size=0.2)
for train, test in cv.split(df_1, groups=df_1['Trial']):
  y = df_1['Value']
  X = df_1.drop('Value', axis=1)
  rfc.fit(X.loc[train], y.loc[train])
  predicted = rfc.predict(X.loc[test])
  actual = y.loc[test]
  r = np.corrcoef(predicted, actual)[0, 1]

In [8]:
r

-0.17822243882789818

In [18]:
import random
random_samp_list = []
index_list = []
for i in range(1,201):
  index_list.append(i)


train_samp_list = random.sample(index_list,k=100)
  
test_list = []
test_list = [x for x in index_list if x not in train_samp_list]


#df_train = df['Trial'] == i
print(train_samp_list)
print(test_list)

[121, 176, 71, 12, 29, 142, 81, 53, 171, 10, 169, 47, 143, 189, 199, 80, 150, 22, 112, 24, 74, 76, 118, 116, 132, 162, 154, 193, 88, 67, 101, 127, 30, 196, 182, 58, 23, 90, 198, 175, 56, 39, 170, 168, 110, 48, 164, 68, 159, 108, 8, 44, 109, 172, 200, 119, 148, 51, 64, 87, 77, 183, 26, 124, 93, 14, 136, 167, 163, 185, 152, 107, 45, 9, 103, 36, 187, 72, 52, 151, 173, 166, 145, 70, 34, 106, 31, 194, 120, 190, 65, 134, 86, 59, 179, 147, 37, 66, 57, 126]
[1, 2, 3, 4, 5, 6, 7, 11, 13, 15, 16, 17, 18, 19, 20, 21, 25, 27, 28, 32, 33, 35, 38, 40, 41, 42, 43, 46, 49, 50, 54, 55, 60, 61, 62, 63, 69, 73, 75, 78, 79, 82, 83, 84, 85, 89, 91, 92, 94, 95, 96, 97, 98, 99, 100, 102, 104, 105, 111, 113, 114, 115, 117, 122, 123, 125, 128, 129, 130, 131, 133, 135, 137, 138, 139, 140, 141, 144, 146, 149, 153, 155, 156, 157, 158, 160, 161, 165, 174, 177, 178, 180, 181, 184, 186, 188, 191, 192, 195, 197]


In [0]:
df_train = df.drop(df.index[train_samp_list])
df_test = df.drop(df.index[test_list])

    
df_x = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)
df_y = df_train.drop(['Subject', 'Block', 'Trial'], axis=1)

In [0]:
y_pd = df_y['Value']
x_pd = df_x

In [22]:
y_pd_predicted = rfc.predict(x_pd)
#formula
#def report(y, y_predicted):
 # conf_mat = confusion_marix(y, y_predicted)
  #report = pd.DataFrame(index=['Value'])
  
print(y_pd_predicted)

[1. 1. 1. ... 3. 3. 3.]
